## Choix du Projet

Estimation du prix d'une habitation via une régression. Dataset: https://www.kaggle.com/datasets/shivachandel/kc-house-data

1.

Début des imports

In [122]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression

data = pd.read_csv("../Data/kc_house_data.csv")

data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180.0,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170.0,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770.0,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050.0,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680.0,0,1987,0,98074,47.6168,-122.045,1800,7503


2.

Nettoyage des données

In [123]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21611 non-null  float64
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [124]:
# il y a 2 valeur manquante, je choisi de les suprimer car cela ne représente que 2 lignes
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21611 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21611 non-null  int64  
 1   date           21611 non-null  object 
 2   price          21611 non-null  float64
 3   bedrooms       21611 non-null  int64  
 4   bathrooms      21611 non-null  float64
 5   sqft_living    21611 non-null  int64  
 6   sqft_lot       21611 non-null  int64  
 7   floors         21611 non-null  float64
 8   waterfront     21611 non-null  int64  
 9   view           21611 non-null  int64  
 10  condition      21611 non-null  int64  
 11  grade          21611 non-null  int64  
 12  sqft_above     21611 non-null  float64
 13  sqft_basement  21611 non-null  int64  
 14  yr_built       21611 non-null  int64  
 15  yr_renovated   21611 non-null  int64  
 16  zipcode        21611 non-null  int64  
 17  lat            21611 non-null  float64
 18  long       

In [125]:
data['date'] = pd.to_datetime(data['date'])


In [126]:
# on regarde si il y a des doublons

duplicate_rows = data[data.duplicated()]
duplicate_rows

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


In [127]:
from sklearn.ensemble import IsolationForest
# Pour suprimer les outliers, je vais me servir de IsolationForest qui retire 5% des valeurs (2.5% des valeur les plus basse, et 2.5% des valeur les plus haute)
# Cela aura comme défault de réduire le dataset, de retirer des valeurs qui ne sont pas forcement abérante, mais je trouve pertinent de faire cela pour du machine learning.
# Je vais faire ceci sur l'ensemble de mon dataset, du moins sur les colonnes que je pense pertinente.

colums = data.select_dtypes('number').drop(columns=['id', 'waterfront', 'view', 'condition', 'grade', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long']).columns
colums

# model = IsolationForest(contamination=0.05)
# outliers = model.fit_predict(data[['price']])
# print(data[outliers == -1].max())
# data[outliers == -1].min()

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [128]:
# création du model
model = IsolationForest(contamination=0.05)

# je fais une boucle sur chaque colonnes selectionnées
for column in colums:
    outliers = model.fit_predict(data[[column]])
    # j'enlève 2.5% des valeurs les plus basses, et 2.5% des plus hautes
    data = data[outliers != -1]

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13670 entries, 0 to 21611
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             13670 non-null  int64         
 1   date           13670 non-null  datetime64[ns]
 2   price          13670 non-null  float64       
 3   bedrooms       13670 non-null  int64         
 4   bathrooms      13670 non-null  float64       
 5   sqft_living    13670 non-null  int64         
 6   sqft_lot       13670 non-null  int64         
 7   floors         13670 non-null  float64       
 8   waterfront     13670 non-null  int64         
 9   view           13670 non-null  int64         
 10  condition      13670 non-null  int64         
 11  grade          13670 non-null  int64         
 12  sqft_above     13670 non-null  float64       
 13  sqft_basement  13670 non-null  int64         
 14  yr_built       13670 non-null  int64         
 15  yr_renovated   13670 non

In [129]:
print(data.max())
data.min()

id                        9842300485
date             2015-05-15 00:00:00
price                      1236000.0
bedrooms                           5
bathrooms                       3.25
sqft_living                     3420
sqft_lot                       39446
floors                           2.0
waterfront                         1
view                               4
condition                          5
grade                             11
sqft_above                    3004.0
sqft_basement                   1130
yr_built                        2015
yr_renovated                    2015
zipcode                        98199
lat                          47.7776
long                        -121.352
sqft_living15                   2870
sqft_lot15                     15625
dtype: object


id                           2800031
date             2014-05-02 00:00:00
price                       146300.0
bedrooms                           2
bathrooms                        1.0
sqft_living                      820
sqft_lot                         696
floors                           1.0
waterfront                         0
view                               0
condition                          1
grade                              5
sqft_above                     790.0
sqft_basement                      0
yr_built                        1900
yr_renovated                       0
zipcode                        98001
lat                          47.1559
long                        -122.475
sqft_living15                   1030
sqft_lot15                      1198
dtype: object

3.

Mise à l'échelle des valeurs

In [130]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13670 entries, 0 to 21611
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             13670 non-null  int64         
 1   date           13670 non-null  datetime64[ns]
 2   price          13670 non-null  float64       
 3   bedrooms       13670 non-null  int64         
 4   bathrooms      13670 non-null  float64       
 5   sqft_living    13670 non-null  int64         
 6   sqft_lot       13670 non-null  int64         
 7   floors         13670 non-null  float64       
 8   waterfront     13670 non-null  int64         
 9   view           13670 non-null  int64         
 10  condition      13670 non-null  int64         
 11  grade          13670 non-null  int64         
 12  sqft_above     13670 non-null  float64       
 13  sqft_basement  13670 non-null  int64         
 14  yr_built       13670 non-null  int64         
 15  yr_renovated   13670 non

In [131]:
from sklearn.preprocessing import MinMaxScaler
# je selectionne les colonnes que je vais scale et celle que je veux garder comme telle
columns_to_scale = ['sqft_living', 'sqft_lot', 'floors', 'view' ,'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'sqft_living15', 'sqft_lot15']
columns_to_keep =  ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'waterfront', 'yr_renovated', 'zipcode', 'lat', 'long']

scaler = MinMaxScaler()
# je fais un DF avec les colonnes scale
scaled_data = pd.DataFrame(scaler.fit_transform(data[columns_to_scale]), columns=columns_to_scale)
# je reset les index pour éviter la création de lignes lors de la concat
data.reset_index(drop=True, inplace=True)
# je concat toute mes colonnes
data = pd.concat([data[columns_to_keep], scaled_data], axis=1)

data.head()


,id,date,price,bedrooms,bathrooms,waterfront,yr_renovated,zipcode,lat,long,...,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,yr_built,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,0,0,98178,47.5112,-122.257,...,0.127845,0.0,0.0,0.5,0.333333,0.176152,0.000000,0.478261,0.168478,0.308588
1,6414100192,2014-12-09,538000.0,3,2.25,0,1991,98125,47.7210,-122.319,...,0.168929,1.0,0.0,0.5,0.333333,0.623306,0.353982,0.443478,0.358696,0.446455
2,2487200875,2014-12-09,604000.0,4,3.00,0,0,98136,47.5208,-122.393,...,0.111071,0.0,0.0,1.0,0.333333,0.117435,0.805310,0.565217,0.179348,0.263534
3,1954400510,2015-02-18,510000.0,3,2.00,0,0,98074,47.6168,-122.045,...,0.190555,0.0,0.0,0.5,0.500000,0.401987,0.000000,0.756522,0.418478,0.437028
4,1321400060,2014-06-27,257500.0,3,2.25,0,0,98003,47.3097,-122.327,...,0.158013,1.0,0.0,0.5,0.333333,0.417796,0.000000,0.826087,0.656522,0.389617


In [132]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13670 entries, 0 to 13669
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             13670 non-null  int64         
 1   date           13670 non-null  datetime64[ns]
 2   price          13670 non-null  float64       
 3   bedrooms       13670 non-null  int64         
 4   bathrooms      13670 non-null  float64       
 5   waterfront     13670 non-null  int64         
 6   yr_renovated   13670 non-null  int64         
 7   zipcode        13670 non-null  int64         
 8   lat            13670 non-null  float64       
 9   long           13670 non-null  float64       
 10  sqft_living    13670 non-null  float64       
 11  sqft_lot       13670 non-null  float64       
 12  floors         13670 non-null  float64       
 13  view           13670 non-null  float64       
 14  condition      13670 non-null  float64       
 15  grade          1367

4.

Mise en place du model de machine learning

In [154]:
from sklearn.model_selection import train_test_split

X = data[['price']]
y = data.select_dtypes('number').drop(columns=['id', 'price', 'yr_renovated', 'lat', 'long', 'zipcode'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.80)

modelLR = LinearRegression().fit(X_train, y_train)

print("Score for the Train dataset :", modelLR.score(X_train, y_train))
print("Score for the Test dataset :", modelLR.score(X_test, y_test))

Score for the Train dataset : 0.0718398838968317
Score for the Test dataset : 0.07232319068854454
